In [2]:
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from datetime import datetime
from time import sleep as para
from time import time as contar
import datetime as dt

import pandas as pd
import PyPDF2
from openpyxl import load_workbook, Workbook
from tkinter import messagebox
from docx import Document
import win32com.client
from tkinter import Tk
import os
import re

### Impressão da tabela

In [13]:
import pandas as pd

df = pd.read_excel(r"C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\ARQUIVOS\obras_2021.xlsx")

contratos_processados = set()  # Crie um conjunto para rastrear os contratos processadosll

for n_contrato in df["CONTRATO N°"]:
    if n_contrato not in contratos_processados:
        print(f'++++++++++++ {n_contrato} ++++++++++++')
        corte_df = df[df['CONTRATO N°'] == n_contrato]
        # display(corte_df)
        t_corte_df = corte_df[['VALOR EMPENHADO (R$)', 'VALOR ANULADO (R$)',
       'VALOR TOTAL (R$)', 'EMPENHO', 'UNIDADE ORÇAMENTARIA',
       'PROGRAMA DE TRABALHO', 'FONTE DE RECURSO', 'NATUREZA DESPESA',
       'CONTA CONTÁBIL']]
        # display(t_corte_df)
        contratos_processados.add(n_contrato)


# print(excel.tolist())
# t_corte_df = corte_df[['VALOR EMPENHADO (R$)', 'VALOR ANULADO (R$)',
#        'VALOR TOTAL (R$)', 'EMPENHO', 'UNIDADE ORÇAMENTARIA',
#        'PROGRAMA DE TRABALHO', 'FONTE DE RECURSO', 'NATUREZA DESPESA',
#        'CONTA CONTÁBIL']]

# display(df)
df.values.tolist()

++++++++++++ 018/2021 ++++++++++++
++++++++++++ 020/2021 ++++++++++++
++++++++++++ 022/2021 ++++++++++++
++++++++++++ 023/2021 ++++++++++++
++++++++++++ 025/2021 ++++++++++++
++++++++++++ 036/2021 ++++++++++++
++++++++++++ 037/2021 ++++++++++++
++++++++++++ 038/2021 ++++++++++++
++++++++++++ 039/2021 ++++++++++++
++++++++++++ 040/2021 ++++++++++++
++++++++++++ Emenda Parlamenta n° 137/2021 ++++++++++++


[['CONSTRUÇÃO DE TALUDE - RUA BELA VISTA C/ BECO BELA VISTA, S/Nº - CONJUNTO PROMORAR, LOCALIZADA NO MUNICÍPIO DE MANAUS.',
  '018/2021',
  'RDS ENGENHARIA DE OBRAS E EMPREENDIMENTOS IMOBILIÁRIOS EIRELI',
  'PARADA TÉCNICA',
  271269.96,
  271269.96,
  nan,
  nan,
  '2021NE00542',
  '27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA',
  '15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS DE CONTENÇÃO ',
  '01000000 - RECURSOS ORDINÁRIOS',
  '44905117 - OBRAS DE INFRAESTRUTURA ',
  '123210517 - OBRAS DE INFRAESTRUTURA DE REVITALIZAÇÃO, URBANIZAÇÃO '],
 ['CONSTRUÇÃO DE TALUDE - RUA BELA VISTA C/ BECO BELA VISTA, S/Nº - CONJUNTO PROMORAR, LOCALIZADA NO MUNICÍPIO DE MANAUS.',
  '018/2021',
  'RDS ENGENHARIA DE OBRAS E EMPREENDIMENTOS IMOBILIÁRIOS EIRELI',
  'PARADA TÉCNICA',
  271269.96,
  nan,
  32509.05,
  'ANULAÇÃO PARCIAL',
  '2021NE01567',
  '27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA',
  '15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS DE CONTENÇÃO ',
  '01000000 - RECURSOS ORDINÁRIOS',
  '4

### Doc_Contratos

Bloco já executado e funcional. As planilhas de contratos estão na pasta: \\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS

In [3]:
class Doc_Contratos():
    def __init__(self):
        for ano in range(2014,2023):
            # Exemplo de uso
            save = os.path.join(r'\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS', f'contratos de obras - {ano}.xlsx')
            caminho_pasta = os.path.join(r"\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS", f"CONTRATO_{ano}")
            if os.path.exists(save): print(f"Arquivo contratos de obras - {ano}.xlsx já existe!")
            else:
                df = self.ler_arquivos_em_pasta(caminho_pasta, ano)
                info = pd.DataFrame(df).transpose()
                info.to_excel(save, index=True)

    def ler_arquivo_docx(self, caminho_arquivo):
        print(f'+++++++++++ Executando ler_arquivo_docx! ({caminho_arquivo}) +++++++++++')
        doc = Document(caminho_arquivo)
        
        conteudo = ""
        for paragrafo in doc.paragraphs:
            conteudo += paragrafo.text + "\n"
        
        return conteudo

    def ler_arquivo_doc(self, caminho_arquivo_doc, word):
        print(f'+++++++++++ Executando ler_arquivo_doc! ({caminho_arquivo_doc}) +++++++++++')
        doc = word.Documents.Open(caminho_arquivo_doc)
        
        # Extrai o texto do documento .doc
        texto = doc.Content.Text
        
        # Fecha o documento e o aplicativo Word
        doc.Close()
        
        return texto

    def extraindo_informacoes(self, texto, arquivo):
        # print(texto.replace('\r','\n'))
        print(f'+++++++++++ Executando extraindo_informacoes! {arquivo} +++++++++++')
        # Extrair número do contrato
        # numero_contrato = re.search(r"Contrato nº\.\s+(\d+).*\r", texto).group(1)
        numero_contrato = re.search(r"Contrato n[º|°]\.?\s*(\d+).*\r", texto).group(1)

        # Extrair data do contrato
        data_contrato = re.search(r"celebrado [em|em\s|\s]*(\d{1,2}.?[/|\.]\d{1,2}[/|\.]\d{4})", texto).group(1)
        # data_contrato = datetime.strptime(data_contrato, "%d.%m.%Y")

        # Extrair nome da empresa
        # nome_empresa = re.search(r"[empresa\s+|consórcio](.+)[\r]?[3.OBJETO]?", texto).group(1)
        # nome_empresa = re.search(r"PARTES: (.+)\r", texto).group(1)
        nome_empresa = re.search(r"PARTES:\s*(.+)\r", texto).group(1)

        # Extrair objeto/obra
        # objeto_obra = re.search(r"OBJETO:\s(.+), [de acordo|conformeManaus/AM]", texto).group(1)
        # objeto_obra = re.search(r"OBJETO:\s*(.+), [de acordo|conforme|Manaus/AM]", texto).group(1)
        objeto_obra = re.search(r"OBJETO[:]?\s*(.+), [de acordo|conformeManaus/AM]", texto).group(1)

        # Extrair valor global
        valor_global = re.search(r"[R\$|VALOR GLOBAL:] ([\d.,]+)", texto).group(1)

        # Extrair nota de empenho
        nota_empenho = re.findall(r"[nº|n|n°][\.|\s]+(\b\d{5}\b)", texto)
        'Empenhos nº. 01136 e nº 01138,'

        # Extrair prazo
        # prazo = re.search(r"PRAZO:\D+(\d+)\D+", texto).group(1)

        # Imprimir as informações extraídas
        # print("Número do contrato:", numero_contrato)
        # print("Data do contrato:", data_contrato)
        # print("Empresa:", nome_empresa)
        # print("Objeto/Obra:", objeto_obra)
        # print("Valor global:", valor_global)
        # print("Nota de empenho:", nota_empenho)
        # print("Prazo:", prazo)

        return (numero_contrato, data_contrato, nome_empresa, objeto_obra, valor_global, tuple(nota_empenho))

    def ler_arquivos_em_pasta(self, caminho_pasta, ano):
        print(f'+++++++++++ Extraindo ler_arquivos_em_pasta! ({ano}) +++++++++++')
        df = {}
        arquivos = os.listdir(caminho_pasta)
        word = win32com.client.Dispatch("Word.Application")
        
        # print('Extraindo ler_arquivos_em_pasta!')
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(caminho_pasta, arquivo)
            
            if '.doc' in arquivo and os.path.isfile(caminho_arquivo): texto = self.ler_arquivo_doc(caminho_arquivo, word)
            else: texto = self.ler_arquivo_docx(caminho_arquivo=caminho_arquivo)

            informacao = self.extraindo_informacoes(texto, arquivo)
            df[arquivo] = informacao
        
        word.Quit()

        return df

    def contrato_completo(self):
        print(f'+ + + + + Executando contrato_completo! + + + + +')
        word = win32com.client.Dispatch("Word.Application")
        for ano in range(2014,2023):
            caminho_excel = os.path.join(r'\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS', f'contratos de obras - {ano}.xlsx')
            caminho_pasta = os.path.join(r"\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS", f"CONTRATO_{ano}")
            print(f'+ + + + + Abrindo a planilha {caminho_excel}! + + + + +')
            txt_contrato = []
            df = pd.read_excel(caminho_excel)
            arquivos = df[df.columns[0]]
        
        # print('Extraindo ler_arquivos_em_pasta!')
            for arquivo in arquivos:
                # print(f'+ + + + + Abrindo o contrato {arquivo}! + + + + +')
                caminho_arquivo = os.path.join(caminho_pasta, arquivo)
                
                if '.doc' in arquivo and os.path.isfile(caminho_arquivo): texto = self.ler_arquivo_doc(caminho_arquivo, word)
                else: texto = self.ler_arquivo_docx(caminho_arquivo=caminho_arquivo)

                txt_contrato.append(texto.replace('\r\r','\n'))

            df['Descrição dos Contratos'] = txt_contrato
            df.to_excel(caminho_excel, index=False)
            
        word.Quit()


### Detalhando NEs

In [4]:
class Navegacao():
    def __init__(self):

        servico = Service(ChromeDriverManager().install())

        print("Fechar a janela!")
        janela = Tk()
        ask = messagebox.askyesno("Esconder Navegador", "Deseja esconder o navegador?")
        janela.lift
        if ask:
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")
            navegador = webdriver.Chrome(options=options, service=servico)
        else: navegador = webdriver.Chrome(service=servico)
        janela.destroy()
        print("Janela fechada!")
        self.vars = {'ignored_exceptions': (NoSuchElementException,StaleElementReferenceException), 
                    'senha': f'Jaspekde27.{dt.date.today().month*3}'}
        self.navegador = navegador

    def esperar_elementos_carregarem(navegador, tipo_elemento: str, nome_elemento: str, varios_elementos: bool = False, tempo_maximo_espera: int = 30):
        """Espera um ou vários elementos carregarem na página

        Args:
            tipo_elemento (str): tipo do elemento
            nome_elemento (str): nome do elemento
            varios_elementos (bool): procurar mais de um elemento
            tempo_maximo_espera (int): tempo determinado para executar a espera, em segundos. Padrão 30 (0,5 min).
        """    
        tempo_inicial = contar() 
        
        if varios_elementos:
            elementos = WebDriverWait(navegador, tempo_maximo_espera, ignored_exceptions=(NoSuchElementException,StaleElementReferenceException))\
                            .until(expected_conditions.presence_of_all_elements_located((tipo_elemento, nome_elemento)))
        else:
            elemento = WebDriverWait(navegador, tempo_maximo_espera, ignored_exceptions=(NoSuchElementException,StaleElementReferenceException))\
                            .until(expected_conditions.presence_of_element_located((tipo_elemento, nome_elemento)))
            
        tempo_final = contar()

        if varios_elementos:
            print(f'>>> Elementos "{nome_elemento}" encontrados em {round(tempo_final - tempo_inicial, 2)}s! (len({len(elementos)}))')
            return elementos
        else:
            if len(elemento.text) > 20:
                elemento_texto_resumido = elemento.text[:20]
            else:
                elemento_texto_resumido = elemento.text
                
            print(f'>>> Elemento "{elemento_texto_resumido}" encontrado em {round(tempo_final - tempo_inicial, 2)}s!')
            return elemento
        
    def LISNE(self, NE, afim):
        print('\n=============== Executando LISNE! ===============')
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        if 'NE' in NE: NE = NE.split("NE")[1]
        lisne_link = f'http://afim2.manaus.am.gov.br/AfimPRD{afim}/' + '/Lisne.do'
        self.navegador.get(lisne_link)

        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        self.navegador.find_element(By.NAME, 'cboUnidadeGestora').send_keys('270101 - Secretaria Municipal de Infraestrutura')
        self.navegador.find_element(By.ID, 'mkdNumeroNE').clear()
        self.navegador.find_element(By.ID, 'mkdNumeroNE').send_keys(NE)
        try: self.navegador.find_element(By.ID, 'btnProcurar').click()
        except: self.navegador.find_element(By.CSS_SELECTOR, '#IMGprocurar').click()

        para(2)
        try: 
            elemento = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'tr td.readCell')
            ActionChains(self.navegador).double_click(elemento).perform()

        except:
            self.esperar_elementos_carregarem(self.navegador, By.CSS_SELECTOR, '[class="btn btn-outline-primary waves-effect waves-light"]').click()

        para(2)
        # navegador.switch_to.window(navegador.window_handles[-1])

        fonte_de_recurso = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtFonte"]').get_attribute('value')
        natureza_despesa = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtNaturezaDespesa"]').get_attribute('value')
        empenho_original = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtNEOriginal"]').get_attribute('value')
        processo = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtProcesso"]').get_attribute('value')
        valor = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtValor"]').get_attribute('value')

        self.navegador.find_element(By.CSS_SELECTOR, '[data-dismiss="modal-empenho"]').click()

        return (NE, fonte_de_recurso, natureza_despesa, empenho_original, processo, valor)
    
    def NL_detalhada(self, obj_NL):
        """Pesquisa a Nota de Lançamento desejada para retirar as informações

        Args:
            obj_NL (_type_): Objeto Selenium da Nota de Lançamento

        Returns:
            _type_: Tupla com as informações extraidas
        """
        if not 'NL' in obj_NL.text: 
            print(f'\n=============== Impossível executar! Não é uma NL ({obj_NL.text}) ===============')
            return ('', '', '', '', '', '','')
        
        
        print(f'\n=============== Executando a NL ({obj_NL.text}) ===============')
        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        elemento = obj_NL
        ActionChains(self.navegador)\
        .double_click(elemento)\
        .perform()

        para(1)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        
        credor_empresa = self.esperar_elemento(By.CSS_SELECTOR, 'tr td input[name="txtCredor"]').get_attribute('value')
        obs = self.esperar_elemento(By.CSS_SELECTOR, 'textarea[name="txtObservacao"]').text
        NE = self.esperar_elemento(By.CSS_SELECTOR, 'input[name="txtNuNe"').get_attribute('value')
        natureza_despesa = self.esperar_elemento(By.CSS_SELECTOR, 'input[name="txtRdNaturezaDespesa"]').get_attribute('value')
        doc = self.esperar_elemento(By.CSS_SELECTOR, 'input[name="txtRdNuDocmento"').get_attribute('value')
        processo = self.esperar_elemento(By.CSS_SELECTOR, 'input[name="txtRdNuProcesso"').get_attribute('value')
        
        self.navegador.close()
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        
        return (obj_NL.text, credor_empresa, obs, NE, natureza_despesa, doc, processo)
    
    def entra_AFIM(self, afim):
        """Abre o AFIM

        Args:
            afim (_type_): Ano no qual o FIM irá abrir
        """
        print(f'\n=============== Executando entra_AFIM {afim}! ===============')
        
        if afim <= 2018:
            self.navegador.get(f'http://afim2.manaus.am.gov.br/AfimPRD{afim}/')
            self.navegador.find_element(By.NAME, 'user').send_keys('03441042220')
            self.navegador.find_element(By.NAME, 'password').send_keys(self.vars["senha"])
            self.navegador.find_element(By.NAME, 'enviar').click()

        #AFIM DE 2019
        if afim == 2019:
            self.navegador.get('https://afim1.manaus.am.gov.br/AfimPRD2019/')
            self.navegador.find_element(By.NAME, 'user').send_keys('034.410.422-20')
            self.navegador.find_element(By.NAME, 'password').send_keys(self.vars["senha"])
            
            para(2)
            for w in self.navegador.window_handles:
                self.navegador.switch_to.window(w)
                if 'mensagem' in self.navegador.current_url: self.navegador.close()
                elif str(afim) in self.navegador.current_url: pass
                else: self.navegador.close()

        if afim >= 2020:
            self.navegador.get(f'https://afim1.manaus.am.gov.br/AfimPRD{afim}/')
            self.navegador.find_element(By.NAME, 'user').send_keys('034.410.422-20')
            self.navegador.find_element(By.NAME, 'password').send_keys(self.vars["senha"])
            
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        print(f'=============== Finalizando entra_AFIM {afim}! ===============\n')
        para(2)

    def RELSALEMP(self, NE, afim):
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        if 'NE' in NE: NE = NE.split("NE")[1]
        lisne_link = f'http://afim2.manaus.am.gov.br/AfimPRD{afim}/' + '/Relsalemp.do'
        self.navegador.get(lisne_link)

        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        self.navegador.find_element(By.CSS_SELECTOR, 'option[value="270101"]').click()
        self.navegador.find_element(By.CSS_SELECTOR, '[name="numeroNE"]').send_keys(NE)
        self.navegador.find_element(By.CSS_SELECTOR, '#IMGimprimir').click()
        
        # Espere até que a aba/janela fechada seja considerada obsoleta
        WebDriverWait(self.navegador, 10).until(expected_conditions.staleness_of(self.navegador.find_element(By.XPATH, "//body")))

        while os.path.isfile(r"C:\Users\luan.pinto\Downloads\SomeReport.pdf") or os.path.isfile(r"C:\Users\luan.pinto\Downloads\RELEXTRATOEMPENHO.pdf"):
            self._mensagem('Apague, renomeie ou mova os arquivos', 'Entre na pasta de Downloads e apague ou renomeie os arquivos "doc.pdf" e "RelBalanceteMensal.xls", para que o programa possa executar sem erros!')


In [6]:
# extrato_empenho = r"C:\Users\luan.pinto\Downloads\SomeReport.pdf"
extrato_empenho = r"C:\Users\luan.pinto\Downloads\RELEXTRATOEMPENHO.pdf"
ano = 2022

'''Capturando o texto do arquivo'''
with open(extrato_empenho, "rb") as arquivo:
    # Criar um objeto PDFReader
    leitor_pdf = PyPDF2.PdfReader(arquivo)

    # Obter o número total de páginas do PDF
    num_paginas = leitor_pdf.pages

    texto = ''

    # Percorrer as páginas do PDF
    for num_pagina in range(len(num_paginas)):
        # Obter o conteúdo de texto da página
        pagina = leitor_pdf.pages[num_pagina]
        texto += pagina.extract_text()

'''Separando as NLs em conjuntos'''
linhas_em_liquidacao = [linha for linha in texto.splitlines() if re.search(r"^Em Liquidação (.*)$", linha)]
linhas_liquidacao = [linha for linha in texto.splitlines() if re.search(r"^Liquidação (.*)$", linha)]
linhas_anula_em_liquidacao = [linha for linha in texto.splitlines() if re.search(r"^Anula Em Liquidação (.*)$", linha)]

NE_original = {'Em Liquidação': {}, 'Liquidação': {}, 'Anula Em Liquidação': {}}
print("Linhas com ocorrências de 'Em Liquidação':")
for linha in linhas_em_liquidacao:
    print(linha)
    linha = linha.split(' ')
    NL = linha.pop(linha.index([i for i in linha if 'NL' in i.upper()][0])) # Setando a variavel de NL

    NE_original['Em Liquidação'][NL] = [int((([i for i in linha if ',' in i][0]).replace('.','')).replace(',',''))/100] # Capturando o valor da NL
    NE_original['Em Liquidação'][NL].append([i for i in linha if '/' in i][0]) # Capturando a data da NL
    NE_original['Em Liquidação'][NL].append([i for i in linha if re.search(r'\d{5}', i)][0]) # Capturando o evento da NL

print("\nLinhas com ocorrências de 'Liquidação':")
for linha in linhas_liquidacao:
    print(linha)
    linha = linha.split(' ')
    NL = linha.pop(linha.index([i for i in linha if 'NL' in i.upper()][0])) # Setando a variavel de NL

    NE_original['Liquidação'][NL] = [int((([i for i in linha if ',' in i][0]).replace('.','')).replace(',',''))/100] # Capturando o valor da NL
    NE_original['Liquidação'][NL].append([i for i in linha if '/' in i][0]) # Capturando a data da NL
    NE_original['Liquidação'][NL].append([i for i in linha if re.search(r'\d{5}', i)][0]) # Capturando o evento da NL

print("\nLinhas com ocorrências de 'Anula Em Liquidação':")
for linha in linhas_anula_em_liquidacao:
    print(linha)
    linha = linha.split(' ')
    NL = linha.pop(linha.index([i for i in linha if 'NL' in i.upper()][0])) # Setando a variavel de NL

    NE_original['Anula Em Liquidação'][NL] = [int((([i for i in linha if ',' in i][0]).replace('.','')).replace(',',''))/100] # Capturando o valor da NL
    NE_original['Anula Em Liquidação'][NL].append([i for i in linha if '/' in i][0]) # Capturando a data da NL
    NE_original['Anula Em Liquidação'][NL].append([i for i in linha if re.search(r'\d{5}', i)][0]) # Capturando o evento da NL



Linhas com ocorrências de 'Em Liquidação':
Em Liquidação  79.630,04 2022NL00172 21/02/2022 500001
Em Liquidação  73.259,64 2022NL00439 10/03/2022 500001
Em Liquidação  79.630,04 2022NL00444 10/03/2022 500001
Em Liquidação  42.150,56 2022NL01481 23/05/2022 500001
Em Liquidação  53.079,32 2022NL02491 12/07/2022 500001
Em Liquidação  53.079,32 2022NL02737 21/07/2022 500001
Em Liquidação  4.366,96 2022NL03194 10/08/2022 500001
Em Liquidação  4.366,96 2022NL03202 10/08/2022 500001
Em Liquidação  4.366,96 2022NL03242 12/08/2022 500001
Em Liquidação  7.657,62 2022NL03880 15/09/2022 500001

Linhas com ocorrências de 'Liquidação':
Liquidação 15/03/2022 2022NL00510  79.630,04 510013
Liquidação 30/05/2022 2022NL01579  42.150,56 510013
Liquidação 20/09/2022 2022NL04110  7.657,62 510013

Linhas com ocorrências de 'Anula Em Liquidação':
Anula Em Liquidação 21/02/2022 505001 2022NL00334  79.630,04
Anula Em Liquidação 10/03/2022 505001 2022NL00441  73.259,64
Anula Em Liquidação 12/07/2022 505001 2022N

In [8]:
data = re.search(r"Data: (.*)\n", texto).group(1)
natureza_despesa = re.search(r"Nat. Despesa: (.*)UO:", texto).group(1)
credor = re.search(r"Credor: (.*)\n", texto).group(1)
processo = re.search(r"Processo: (.*)\n", texto).group(1)
NLs = set(re.findall(r"{}NL\d+".format(ano), texto))
OB = set(re.findall(r"{}OB\d+".format(ano), texto))
NE = set(re.findall(r"{}NE\d+".format(ano), texto))

print()
print('Data:',data)
print('Natureza da despesa:',natureza_despesa)
print('Credor:',credor)
print('N° de Processo:',processo)
print('N° das OBs:',OB)
print('N° das NEs:',NE)
print('N° das NLs:',NLs)


Data: 03/01/2022
Natureza da despesa: 44905193 - Reformas, Benfeitorias Ou Melhoria
Credor:  84462209000167 - SYRIA ENGENHARIA E CONSTRUÇÃO EIRELI
N° de Processo: 2021/20000/20031/0/001211
N° das OBs: {'2022OB00183', '2022OB00461', '2022OB01344'}
N° das NEs: {'2022NE01393', '2022NE00271', '2022NE01249'}
N° das NLs: {'2022NL02491', '2022NL02737', '2022NL01579', '2022NL03195', '2022NL03002', '2022NL03880', '2022NL04110', '2022NL00444', '2022NL01481', '2022NL00441', '2022NL03652', '2022NL03194', '2022NL00172', '2022NL03242', '2022NL03235', '2022NL03202', '2022NL00334', '2022NL00439', '2022NL00510', '2022NL02734'}


In [6]:
ano = 2014

driver = Navegacao()
driver.entra_AFIM(ano)
driver.LISNE('01136',ano)

Fechar a janela!
Janela fechada!

=============== Executando entra_AFIM 2014! ===============
=============== Finalizando entra_AFIM 2014! ===============


=============== Executando LISNE! ===============


AttributeError: 'Navegacao' object has no attribute 'find_elements'

In [ ]:
NE = '01136'

driver.navegador.switch_to.window(driver.navegador.window_handles[-1])
if 'NE' in NE: NE = NE.split("NE")[1]
lisne_link = f'http://afim2.manaus.am.gov.br/AfimPRD{ano}/' + '/Relsalemp.do'
driver.navegador.get(lisne_link)

In [ ]:
driver.navegador.find_element(By.CSS_SELECTOR, 'option[value="270101"]').click()
driver.navegador.find_element(By.CSS_SELECTOR, '[name="numeroNE"]').send_keys(NE)
driver.navegador.find_element(By.CSS_SELECTOR, '#IMGimprimir').click()
